In [1]:
import scipy.io
import h5py
import numpy as np
from open_matlab import matlab2python
from timeit import default_timer as timer

In [3]:
# print the available fields 
fields = matlab2python(1, 'ct20_test', None)
print(fields, len(fields))


['A', 'A_pstd', 'SDa', 'SDb', 'SDc', 'SDpreF', 'asynch2nd', 'asynchAlloc', 'c', 'c_pstd', 'df', 'dfCrit', 'dist2Reclust', 'dist2centFAll', 'dist2centFAsynch', 'dist2centFSynch', 'dist2centFSynch2nd', 'dist2centPhyAll', 'dist2centPhyAsynch', 'dist2centPhySynch', 'dist2centPhySynch2nd', 'distFromLast', 'frame', 'frameRelat', 'permanence', 's', 'stimNum', 'synID', 'synSigma', 'synch', 'tTrcx', 'tTrcxy', 'tTrcxyAft1', 'tTrcxyAft2', 'tTrcxyBef', 'tTrcy', 'timeFromLast', 'traceDt', 'traceInt', 'traceIntPeak', 'traceIntrat1', 'traceIntrat2', 'traceLoc', 'traceLocN', 'traceLocPeak', 'traceLocrat1', 'traceLocrat2', 'val', 'x', 'x_pstd', 'y', 'y_pstd'] 52


In [4]:
#use the desired fields
data = matlab2python(0, 'ct20_test',  ['synID','x','y', 'timeFromLast'])
print(data.shape)

OPENING : /home/gerard/nextcloud/analysis/localization/summary/ct20_test.mat
0 synID
1 x
2 y
3 timeFromLast
(68238, 4)


In [5]:
print(data[0])
print(data[0,0])
print(data[:,0])

[5.85060000e+04 2.10154416e+01 2.00624993e+01            nan]
58506.0
[5.85060000e+04 5.85060000e+04 5.85060000e+04 ... 1.11021137e+09
 1.11021137e+09 1.11021137e+09]


In [2]:
#use ALL the fields
data_10 = matlab2python(0, 'ct20_test',  fields)
print(data_10.shape)

OPENING : /home/gerard/nextcloud/analysis/localization/summary/ct20_test.mat
0 dist2centFAll
long field, not included  tTrcx
(68238,)


In [4]:
data_all = matlab2python(0, 'ct20_test',  ['traceLoc'])
print(data_all.shape)

OPENING : /home/gerard/nextcloud/analysis/localization/summary/ct20_test.mat
long field, not included  traceLoc
(0,)


In [126]:
start = timer()
# Replace 'file_path' with the path to your MATLAB file.
file_path = '/media/data/nextcloud/analysis/summary/ct20_test.mat' #/home/gerard/analysis/from_matlab/summary/ct20_test.mat'
fields2call = ['synID','x','y', 'timeFromLast']
file_name = '/ct20_test/allEv/'

# Open the MATLAB file using h5py
with h5py.File(file_path, 'r') as file:
    # Access the 'allEv' dataset
    allEv_data = np.array([])#[]

    for idx, name in enumerate(fields2call):

        allEv_dataset = file[file_name + name]

        # Convert the references to strings
        allEv_references = [ref.item() for ref in allEv_dataset]

        carrier =  []
        for ref in allEv_references:
            item = file[ref]
            carrier.append(item[:])
        carrier = np.squeeze(np.array(carrier))

        if allEv_data.size == 0:
           allEv_data = np.append(allEv_data, carrier)

        else:
            allEv_data = np.column_stack((allEv_data, carrier))
        
end = timer()
print(end - start)

24.65767366199725


In [121]:
print(type(allEv_data))
print(np.shape(allEv_data))
print(allEv_data[0,0], allEv_data[0,1], allEv_data[0,2])

<class 'numpy.ndarray'>
(68238, 3)
58506.0 21.01544156606587 20.0624993068723
